In [11]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import requests
from bs4 import BeautifulSoup
import time
import lxml
import re

# Parsing XML Files from ProPublica

Last updated: 6/1/23 by Christopher Lum

### Methods

`scrape_website(url, attempts=0)`
- Takes a URL and returns the response object if successful. Retries three times on failure

`info_grabber(fp, url=True)`
- Takes the URL/filepath of an XML file. The `url` parameter indicates whether the given filepath is a URL or local
- Returns a dictionary of the row information for section VIII (along with a couple other columns)
- Older (prior to 2013) XML files have different tags and thus can't be read. Results in a row of all zeroes and `'Not Found'` in the `ba` column
- Of successfully scraped columns, has ~1.5% error rate for rows (0.13% of cells are inaccurate), only in the `Fundraising Events` column due to inconsistent labeling in XMLs
    - This error rate is lower than the human error rate
- In 2013, they had a different way of tagging zip codes, so all 2013 zips will be 0

`contact_grabber(fp, url=True)`
- Returns a dataframe of each contact in the 990 form, according to section VII
- Incomplete and edge cases aren't tested fully, as we didn't need the function.

`get_xmls(ein)`
- Takes the EIN of a company and returns a list of the URLs for each XML file that is linked on the ProPublica page for that given company

`grabber(eins, verbose=False, clean=False)`
- Takes a list of EINs and uses `get_xmls()` to find each XML file, then uses `info_grabber()` to scrape each XML for the relevant information.
- Returns a pandas DataFrame, doesn't automatically save as csv.
- Takes around 22 minutes on my laptop for 71 businesses
- The `clean` parameter automatically cleans the dataframe by removing empty rows

`find_errors(info)`
- Data columns are supposed to add up to the total, if it doesn't this method will find it
- Takes a dataframe of 990 rows from `grabber` and returns a tuple of incides that didn't properly grab all of the data
    - Preferably indiced on `EIN_YEAR`

#### `scrape_website`

In [12]:
def scrape_website(url, attempts=0):
    # Send GET request to the website
    response = requests.get(url)
    time.sleep(1)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        if attempts > 0:
            print('Success')
        return response
    elif attempts < 3:
        print("Error:", response.status_code, "Trying again.")
        time.sleep(1)
        return scrape_website(url, attempts + 1)
    else:
        # Handle the request error
        print("Error: ", response.status_code, 'Failed on:', url)
        return None


#### `info_grabber`

In [13]:
def info_grabber(fp, url=True):
    '''
    Returns a dictionary that can be a row in a dataframe of the important info from a given XML url

    :param fp: Filepath of xml
    :param url: Whether a filepath is local or url
    '''
    if url:
        xml = scrape_website(fp)
        xml.encoding = 'UTF-8'
        xml = xml.text.strip()
    else:
        try:
            with open(fp, 'r') as file:
                xml = file.read()
        except FileNotFoundError:
            print(f"File not found: {fp}")
            return dict()
    if xml is None:
        return dict()
    
    soup = BeautifulSoup(xml, 'xml')

    row = dict()
    row['ba'] = soup.find('BusinessName')
    row['EIN'] = soup.find('EIN')
    row['Tax Year'] = soup.find('TaxPeriodBeginDt')
    row['Location (Zipcode)'] = soup.find('USAddress')
    row['Federate Campaigns'] = soup.find('FederatedCampaignsAmt')
    row['Membership Dues'] = soup.find('MembershipDuesAmt')
    row['Fundraising Events'] = soup.find('ContriRptFundraisingEventAmt')
    row['Related Organizations'] = soup.find('RelatedOrganizationsAmt')
    row['Government Grants'] = soup.find('GovernmentGrantsAmt')
    row['All Other Contributions'] = soup.find('AllOtherContributionsAmt')
    row['Noncash Contributions'] = soup.find('NoncashContributionsAmt')
    row['Total'] = soup.find('TotalContributionsAmt')
    # Can make default=0

    if row['Location (Zipcode)'] is not None:
        row['Location (Zipcode)'] = row['Location (Zipcode)'].find('ZIPCd')
    else:
        row['Location (Zipcode)'] = None
        
    row = dict(map(lambda item: (item[0], item[1].text if item[1] is not None else None), row.items()))

    if row['ba'] is not None:
        row['ba'] = row['ba'].strip()
    else:
        row['ba'] = 'Not found'
    if row['Tax Year'] is not None:
        row['Tax Year'] = pd.Timestamp(row['Tax Year']).year
    else:
        row['Tax Year'] = np.nan

    row = dict(map(lambda item: (item[0], item[1] if item[1] is not None else 0), row.items()))

    return row

#### `contact_grabber`

Old, might not work

In [14]:
def contact_grabber(fp, url=True):
    '''
    Return a dataframe of the important names and their positions in an organization for each year

    :param fp: filepath of the xml file
    :param url: Whether the fp is a url or not. Default = True
    '''
    if url:
        xml = scrape_website(fp)
    else:
        xml = requests.get(fp)
    if xml is None:
        return dict()
    xml.encoding = 'UTF-8'
    xml = xml.text.strip()
    soup = BeautifulSoup(xml, 'xml')

    people = soup.find_all('Form990PartVIISectionAGrp')
    names = [x.find('PersonNm').text for x in people if x.find('PersonNm') is not None]
    titles = [x.find('TitleTxt').text for x in people if x.find('TitleTxt') is not None]
    try:
        organization = [soup.find('BusinessName').text.strip()] * len(names)
    except AttributeError:
        organization = ['Not found'] * len(names)
    try:
        year = [pd.Timestamp(soup.find('TaxPeriodBeginDt').text).year] * len(names)
    except AttributeError:
        year = [0] * len(names)
        
    return pd.DataFrame({'organization': organization, 'year': year, 'names': names, 'titles': titles})

#### `get_xmls`

In [15]:
def get_xmls(ein):
    '''
    Return a list of xml urls from ProRepublica based on an organization's EIN

    :param ein: Employer Identification number as String or Int
    '''
    url = f'https://projects.propublica.org/nonprofits/organizations/{ein}'
    html = scrape_website(url)
    if html is None:
        return []
    soup = BeautifulSoup(html.text)
    links = soup.find_all('a', class_='action xml')
    base_url = 'https://projects.propublica.org'
    # urls = [base_url + x.get('href') for x in links if x.text == '990']
    urls = [x for x in soup.find_all(class_='action xml') if re.search(r'990\b', x.text)]
    urls = [base_url + x.get('href') if x.name == 'a' else base_url + x.select_one('select.action.xml option[data-href]').get('data-href') for x in urls]
    return urls

#### `grabber`

In [16]:
def grabber(eins, verbose=False, clean=False):
    if type(eins) == int:
        eins = [eins]
    data = []
    # contacts = pd.DataFrame()
    overall_index = 0
    for index, ein in enumerate(eins):
        print(str(index) + (' / ') + str(len(eins)))
        xmls = get_xmls(ein)
        for xml in xmls:
            if verbose:
                print(f'{overall_index} {xml}')
            data.append(info_grabber(xml))
            #contacts = pd.concat([contacts, contact_grabber(xml)])
            overall_index += 1
    print(str(len(eins)) + (' / ') + str(len(eins)))
    info = pd.DataFrame(data)
    if not clean:
        return info

    no_error = info.dropna().reset_index(drop=True)
    no_error.loc[:, 'EIN':'Total'] = no_error.loc[:, 'EIN':'Total'].astype(int)
    no_error.loc[:, 'EIN_YEAR'] = no_error['EIN'].astype(str) + '_' + no_error['Tax Year'].astype(str)
    no_error = no_error.set_index('EIN_YEAR')

    return no_error

#### `find_errors`

In [17]:
def find_errors(info):
    '''
    Takes a dataframe of 990 rows and returns a tuple of the indices where it failed to grab the data properly

    :param info: The dataframe of 990 rows
    '''
    accurate = info[['Federate Campaigns', 'Membership Dues', 'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions']].sum(axis=1) == info['Total']
    return tuple(accurate[~accurate].index)

### Using the `grabber()` method

`grabber()` takes a list of EINs. This was curated from the old "990 Manual Pull" spreadsheet by pasting and splitting the entire `EIN` column and putting it into a set.

In [18]:
all_eins = '''113723093
113723093
113723093
113723093
113723093
237279074
237279074
237279074
237279074
237279074
237334012
237334012
237334012
237334012
237334012
261712580
261712580
261712580
261712580
261712580
261712580
330006089
330006089
330006089
330006089
330215585
330215585
330215585
330215585
330215585
330317950
330317950
330317950
330317950
330433314
330433314
330433314
330433314
330433314
330492304
330492304
330492304
330492304
330492304
330496092
330496092
330496092
330496092
330623634
330623634
330623634
330623634
330623634
331029843
331029843
331029843
331029843
331146733
331146733
331146733
331146733
465055513
465055513
465055513
465055513
942358632
942358632
942358632
942358632
942358632
951644024
951644024
951644024
951644024
951869806
951869806
951869806
951869806
951869806
951869806
951874073
951874073
951874073
951874073
951874073
951874073
951944230
951944230
951944230
951944230
951944230
952039198
952039198
952039198
952039198
952157559
952157559
952157559
952157559
952157559
952648050
952648050
952648050
952648050
952648050
952648050
952653869
952653869
952653869
952653869
952653869
952833205
952833205
952833205
952833205
952833205
952850121
952850121
952850121
952850121
952850121
952880426
952880426
952880426
952880426
952880426
953244085
953244085
953244085
953244085
953244085
953248148
953248148
953248148
953248148
953248148
953302967
953302967
953302967
953302967
953302967
953302967
953315571
953315571
953315571
953315571
953315571
953649525
953649525
953649525
953649525
956379598
956379598
956379598
956379598
953497926
953497926
953497926
953497926
953497926
953497926
953837714
953837714
953837714
953837714
953837714
953140767
953140767
953140767
953140767
951648184
951648184
951648184
951648184
951648184
952693142
952693142
952693142
952693142
952693142
330217339
330217339
330217339
330217339
330217339
330217339
952794253
952794253
952794253
952794253
952794253
952794253
951945256
951945256
951945256
951945256
951945256
270865318
270865318
270865318
270865318
270865318
331146733
331146733
331146733
331146733
331146733
821946283
821946283
263405689
263405689
263405689
263405689
263405689
263405689
330602842
330602842
330602842
330602842
330602842
330902617
330902617
330902617
330902617
330902617
952457354
952457354
952457354
952457354
952457354
330008269
330008269
330008269
330008269
330008269
953031682
953031682
953031682
953031682
953031682
237161267
237161267
237161267
237161267
330497515
330497515
330497515
330497515
330497515
953750738
953750738
953750738
953750738
953750738
953368020
953368020
953368020
953368020
953368020
953368020
953798088
953798088
953798088
953798088
952111196
952111196
952111196
952111196
952949636
952949636
952949636
952949636
952949636
330618893
330618893
953782164
953782164
953782164
953782164
953782164
953782164
272917644
272917644
272917644
272917644
273390797
273390797
273390797
931008369
931008369
931008369
931008369
931008369
952213995
952213995
952213995
952213995
562613191
562613191
562613191
562613191
562613191
952422704
952422704
952422704
952422704
941676390
941676390
941676390
941676390
941676390
550806460
550806460
550806460
550806460
550806460
550806460
330122462
330122462
330122462
330122462
270447059
270447059
270447059
270447059
270447059
330210280
822363154
953138268
953950196'''

In [99]:
wrong_eins = '''952850121 562613191 952457354 330210280 952877102 330433314 204374795 330602842 465055513 263405689 263405689 330602842 330602842 330602842 330433314 951644024 330553621 952653869 330433314 941676390 953302967 941676390 941676390 941676390 465055513 330602842 952850121 330006089 952111196 330433314 562613191 952880426 953244085
'''

In [9]:
eins = list(set(all_eins.split()))
# eins = list(set(wrong_eins.split()))

<b>Warning</b>: `grabber()` takes a long time. The following cell will take about 20 minutes.

In [19]:
info990 = grabber(eins, verbose=True)

0 / 71
0 https://projects.propublica.org/nonprofits/download-xml?object_id=202221319349303917
1 https://projects.propublica.org/nonprofits/download-xml?object_id=202101189349301515
2 https://projects.propublica.org/nonprofits/download-xml?object_id=202010589349300236
3 https://projects.propublica.org/nonprofits/download-xml?object_id=201940249349301019
4 https://projects.propublica.org/nonprofits/download-xml?object_id=201800269349301550
5 https://projects.propublica.org/nonprofits/download-xml?object_id=201710269349300021
6 https://projects.propublica.org/nonprofits/download-xml?object_id=201640329349300984
7 https://projects.propublica.org/nonprofits/download-xml?object_id=201520289349301172
8 https://projects.propublica.org/nonprofits/download-xml?object_id=201400239349301215


KeyboardInterrupt: 

### Checking the missing cells

The following two cells should both be 1.0.

The first checks the conditional probability that if the name is not found in a row, the tax year wasn't found.

The second checks the inverse conditional probability. Sometimes, if the request attempt tried and failed three times, the row will be entirely empty, where the name is `''` and not `'Not Found'`

In [14]:
# Every time the name isn't found, nothing is found
info990.loc[info990['ba'] == 'Not found']['Tax Year'].isna().mean()

1.0

In [15]:
# And every time the tax year isn't found the name isn't found
(info990[['ba', 'Tax Year']].loc[info990['Tax Year'].isna()]['ba'] == 'Not found').mean()

1.0

This cell indicates how many XML files failed to be accessed

In [16]:
info990['ba'].isna().sum()

0

If the following cell returns 0.0, that indicates that every time the tax year is missing, every single financial cell is zero. All of these rows can be dropped and no information is lost.

In [17]:
info990.loc[info990['Tax Year'].isna(), 'Federate Campaigns':].sum(axis=1).sum()

0.0

### Cleaning the raw information

Assuming the prior three checks are all adequate, we can drop rows with any nan values.

In [20]:
no_error = info990.dropna().reset_index(drop=True)
no_error.loc[:, 'EIN':'Total'] = no_error.loc[:, 'EIN':'Total'].astype(int)
no_error.loc[:, 'EIN_YEAR'] = no_error['EIN'].astype(str) + '_' + no_error['Tax Year'].astype(str)
no_error = no_error.set_index('EIN_YEAR')

### Export here!

<b>REMEMBER TO EXPORT</b>

`no_error` is the cleaned dataframe

`info990` is the raw dataframe with missing rows

In [19]:
# no_error.to_csv('no_errors.csv')

In [20]:
# info990.to_csv('full_table.csv')

The following line appends the rows to the bottom of the existing dataframe titled `'no_errors.csv'` instead.

It creates a new CSV and it is recommended to rename it back to `no_errors.csv` in the case it is exported successfully

In [148]:
extended = pd.concat([pd.read_csv('no_errors.csv'), no_error]).drop(columns='Unnamed: 0').reset_index(drop=True).drop_duplicates()
extended.to_csv('no_errors_appended.csv')

## Testing Zone

In [92]:
url = 'https://projects.propublica.org/nonprofits/organizations/263405689'
html = scrape_website(url)
soup = BeautifulSoup(html.text)
links = soup.find_all('a', class_='action xml')
base_url = 'https://projects.propublica.org'
urls = [base_url + x.get('href') for x in links if x.text == '990']

In [93]:
test = [x for x in soup.find_all(class_='action xml') if re.search(r'990\b', x.text)]
test2 = [base_url + x.get('href') if x.name == 'a' else base_url + x.select_one('select.action.xml option[data-href]').get('data-href') for x in test]

In [106]:
info990

,ba,EIN,Tax Year,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
0,ELDERHELP OF SAN DIEGO,952880426,2020.0,92123,0,0,109324,0,574130,1219917,37648,1903371
1,ELDERHELP OF SAN DIEGO,952880426,2019.0,92123,0,0,51009,0,318368,1075276,34594,1444653
2,ELDERHELP OF SAN DIEGO,952880426,2018.0,92123,0,0,53375,0,313363,762231,37263,1128969
3,ELDERHELP OF SAN DIEGO,952880426,2016.0,92123,0,0,42430,0,357117,1052228,0,1451775
4,ELDERHELP OF SAN DIEGO,952880426,2015.0,92123,0,0,40155,0,345108,916212,0,1301475
...,...,...,...,...,...,...,...,...,...,...,...,...
129,CRASH Inc,952653869,2015.0,92108,0,0,0,0,0,12407,0,12407
130,CRASH Inc,952653869,2014.0,92108,0,0,0,0,0,2979,0,2979
131,CRASH Inc,952653869,2013.0,0,0,0,0,0,0,12130,0,12130
132,Not found,952653869,NaN,0,0,0,0,0,0,0,0,0


In [134]:
extended = pd.concat([pd.read_csv('no_errors.csv'), no_error]).drop(columns='Unnamed: 0').reset_index(drop=True).drop_duplicates()

In [138]:
extended.loc[:, 'EIN':'Total'] = extended.loc[:, 'EIN':'Total'].astype(int)

In [144]:
extended = extended.drop_duplicates()

In [2]:
info = pd.read_csv('no_errors.csv')

In [3]:
info = info.drop(columns='Unnamed: 0').set_index('EIN_YEAR')
# accurate = info[['Federate Campaigns', 'Membership Dues', 'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions']].sum(axis=1) == info['Total']

In [4]:
info

,ba,EIN,Tax Year,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
EIN_YEAR,,,,,,,,,,,,
952880426_2020,ELDERHELP OF SAN DIEGO,952880426,2020.0,92123,0,0,109324,0,574130,1219917,37648,1903371
952880426_2019,ELDERHELP OF SAN DIEGO,952880426,2019.0,92123,0,0,51009,0,318368,1075276,34594,1444653
952880426_2018,ELDERHELP OF SAN DIEGO,952880426,2018.0,92123,0,0,53375,0,313363,762231,37263,1128969
952880426_2016,ELDERHELP OF SAN DIEGO,952880426,2016.0,92123,0,0,42430,0,357117,1052228,0,1451775
952880426_2015,ELDERHELP OF SAN DIEGO,952880426,2015.0,92123,0,0,40155,0,345108,916212,0,1301475
...,...,...,...,...,...,...,...,...,...,...,...,...
330553621_2017,THE SALVATION ARMY SAN DIEGO RESIDENCES INC,330553621,2017.0,90275,0,0,0,0,712870,0,0,712870
330553621_2016,THE SALVATION ARMY SAN DIEGO RESIDENCES INC,330553621,2016.0,90802,0,0,0,0,665873,0,0,665873
330553621_2015,THE SALVATION ARMY SAN DIEGO RESIDENCES INC,330553621,2015.0,90802,0,0,0,0,621388,0,0,621388


In [19]:
# tuple(accurate[~accurate].index)

('263405689_2015',
 '952039198_2020',
 '952039198_2013',
 '953302967_2013',
 '953497926_2016',
 '272917644_2017',
 '331146733_2017',
 '953368020_2015',
 '951874073_2015')

In [7]:
find_errors(info)

('263405689_2015',
 '952039198_2020',
 '952039198_2013',
 '953302967_2013',
 '953497926_2016',
 '272917644_2017',
 '331146733_2017',
 '953368020_2015',
 '951874073_2015')